Install when Running for the First Time

In [ ]:
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install plotly

In [ ]:
%conda install -c conda-forge ta-lib

In [ ]:
%pip install nbformat --upgrade

Importing Dependencies

In [1]:
import pandas as pd
import numpy as np
import talib
import matplotlib.pyplot as plt
import pandas_ta as ta

Wrapper Function for reading CSV

In [2]:
def read_csv(file_path):
    df = pd.read_csv(file_path)
    # Convert the 'Date' column to datetime format and set it as the index
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    return df


Calculating Technical Indicators

In [3]:
def calculate_choppiness_index(df, period=14):
    hl_range = df['High'] - df['Low']
    atr = talib.SMA(talib.TRANGE(df['High'], df['Low'], df['Close']), period)
    normalized_range = hl_range / atr
    choppiness_index = talib.SMA(normalized_range, period)
    return choppiness_index

In [4]:
def calculate_indicators(df):
    # High Time Frame (Month)
    df['SMA_20'] = talib.SMA(df['Close'], timeperiod=20)
    df['Choppiness_Index'] = calculate_choppiness_index(df)
    df['SuperTrend'] = talib.SAR(df['High'], df['Low'], acceleration=0.02, maximum=0.2)

    # Mid Time Frame (Weekly)
    df['MACD'], _, _ = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    df['RSI'] = talib.RSI(df['Close'], timeperiod=14)
    df['SMA_50'] = talib.SMA(df['Close'], timeperiod=50)
    df['EMA_20'] = talib.EMA(df['Close'], timeperiod=20)
    df['EMA_10'] = talib.EMA(df['Close'], timeperiod=10)

    # Low Time Frame (Daily)
    df['VWSMA_200'] = df['Close'].rolling(window=200).mean()
    df['VWEMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
    df['VWEMA_20'] = df['Close'].ewm(span=20, adjust=False).mean()
    df['RSI_Daily'] = talib.RSI(df['Close'], timeperiod=14)
    df['ADX'] = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod=14)

    return df


Plotting the Indicators

In [5]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

def plot_chart(df, stock_name):
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, 
                        subplot_titles=("High Time Frame (Month)", 
                                        "Mid Time Frame (Weekly)", 
                                        "Low Time Frame (Daily)"))

    # High Time Frame (Month)
    fig.add_trace(go.Scatter(x=df.index, y=df['Close'], mode='lines', name='Close Price'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['SMA_20'], mode='lines', name='SMA 20'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['Choppiness_Index'], mode='lines', name='Choppiness Index'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['SuperTrend'], mode='lines', name='SuperTrend'), row=1, col=1)

    # Mid Time Frame (Weekly)
    fig.add_trace(go.Scatter(x=df.index, y=df['Close'], mode='lines', name='Close Price'), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['MACD'], mode='lines', name='MACD'), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['RSI'], mode='lines', name='RSI'), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['SMA_50'], mode='lines', name='SMA 50'), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['EMA_20'], mode='lines', name='EMA 20'), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['EMA_10'], mode='lines', name='EMA 10'), row=2, col=1)

    # Low Time Frame (Daily)
    fig.add_trace(go.Scatter(x=df.index, y=df['Close'], mode='lines', name='Close Price'), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['VWSMA_200'], mode='lines', name='VWSMA 200'), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['VWEMA_50'], mode='lines', name='VWEMA 50'), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['VWEMA_20'], mode='lines', name='VWEMA 20'), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['RSI_Daily'], mode='lines', name='RSI Daily'), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['ADX'], mode='lines', name='ADX'), row=3, col=1)

    fig.update_layout(title=f'Stock: {stock_name}', 
                      xaxis_title='Date', 
                      yaxis_title='Price/Indicator',
                      height=800, 
                      width=1000)
    fig.show()



In [6]:
def process_stock(monthly_file, weekly_file, daily_file, stock_name):
    
    # Read CSV files
    df_monthly = read_csv(monthly_file)
    df_weekly = read_csv(weekly_file)
    df_daily = read_csv(daily_file)

    # Calculate technical indicators
    df_monthly = calculate_indicators(df_monthly)
    df_weekly = calculate_indicators(df_weekly)
    df_daily = calculate_indicators(df_daily)

    # Plot charts
    plot_chart(df_monthly, stock_name + ' - Monthly')
    plot_chart(df_weekly, stock_name + ' - Weekly')
    plot_chart(df_daily, stock_name + ' - Daily')


In [7]:
import pandas as pd
import talib
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def read_csv(file_path):
    df = pd.read_csv(file_path)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    return df

def calculate_monthly_indicators(df):
    df['SMA_20'] = talib.SMA(df['Close'], timeperiod=20)
    df['Choppiness_Index'] = calculate_choppiness_index(df)
    df['SuperTrend'] = talib.SAR(df['High'], df['Low'], acceleration=0.02, maximum=0.2)
    return df[['SMA_20', 'Choppiness_Index', 'SuperTrend']]

def calculate_weekly_indicators(df):
    df['MACD'], _, _ = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    df['RSI'] = talib.RSI(df['Close'], timeperiod=14)
    df['SMA_50'] = talib.SMA(df['Close'], timeperiod=50)
    df['EMA_20'] = talib.EMA(df['Close'], timeperiod=20)
    df['EMA_10'] = talib.EMA(df['Close'], timeperiod=10)
    return df[['MACD', 'RSI', 'SMA_50', 'EMA_20', 'EMA_10']]

def calculate_daily_indicators(df):
    df['VWSMA_200'] = df['Close'].rolling(window=200).mean()
    df['VWEMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
    df['VWEMA_20'] = df['Close'].ewm(span=20, adjust=False).mean()
    df['RSI_Daily'] = talib.RSI(df['Close'], timeperiod=14)
    df['ADX'] = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod=14)
    return df[['VWSMA_200', 'VWEMA_50', 'VWEMA_20', 'RSI_Daily', 'ADX']]

def calculate_choppiness_index(df, period=14):
    hl_range = df['High'] - df['Low']
    atr = talib.SMA(talib.TRANGE(df['High'], df['Low'], df['Close']), period)
    normalized_range = hl_range / atr
    choppiness_index = talib.SMA(normalized_range, period)
    return choppiness_index

def plot_trend(df, indicators, stock_name, timeframe):
    fig = make_subplots(rows=len(indicators), cols=1, shared_xaxes=True, 
                        subplot_titles=indicators)

    for i, indicator in enumerate(indicators):
        fig.add_trace(go.Scatter(x=df.index, y=df[indicator], mode='lines', name=indicator), row=i+1, col=1)

    fig.update_layout(title=f'Stock: {stock_name} - {timeframe} Trends', 
                      xaxis_title='Date', 
                      yaxis_title='Indicator Value',
                      height=800, 
                      width=1000)
    fig.show()

def process_stock(monthly_file, weekly_file, daily_file, stock_name):
    df_monthly = read_csv(monthly_file)
    df_weekly = read_csv(weekly_file)
    df_daily = read_csv(daily_file)

    monthly_indicators = calculate_monthly_indicators(df_monthly)
    weekly_indicators = calculate_weekly_indicators(df_weekly)
    daily_indicators = calculate_daily_indicators(df_daily)

    plot_trend(monthly_indicators, ['SMA_20', 'Choppiness_Index', 'SuperTrend'], stock_name, 'Monthly')
    plot_trend(weekly_indicators, ['MACD', 'RSI', 'SMA_50', 'EMA_20', 'EMA_10'], stock_name, 'Weekly')
    plot_trend(daily_indicators, ['VWSMA_200', 'VWEMA_50', 'VWEMA_20', 'RSI_Daily', 'ADX'], stock_name, 'Daily')


In [8]:
process_stock('/home/tanishpatel01/Desktop/Stock_Market_Data/ADANIENT_Monthly.csv', '/home/tanishpatel01/Desktop/Stock_Market_Data/ADANIENT_Weekly.csv', '/home/tanishpatel01/Desktop/Stock_Market_Data/ADANIENT_Daily.csv', 'ADANIENT')

In [9]:
def calculate_monthly_indicators(df):
    df['SMA_20'] = talib.SMA(df['Close'], timeperiod=20)
    df['Choppiness_Index'] = calculate_choppiness_index(df)
    # Calculate SuperTrend separately
    df['SuperTrend'] = calculate_super_trend(df)
    return df[['SMA_20', 'Choppiness_Index', 'SuperTrend']]

def calculate_super_trend(df, atr_period=14, multiplier=2):
    high = df['High']
    low = df['Low']
    close = df['Close']
    atr = talib.ATR(high, low, close, timeperiod=atr_period)
    basic_upper_band = (high + low) / 2 + multiplier * atr
    basic_lower_band = (high + low) / 2 - multiplier * atr
    super_trend = pd.Series(index=df.index)
    super_trend.iloc[0] = close.iloc[0]
    for i in range(1, len(super_trend)):
        if close.iloc[i - 1] <= basic_upper_band.iloc[i - 1]:
            super_trend.iloc[i] = max(basic_upper_band.iloc[i], super_trend.iloc[i - 1])
        else:
            super_trend.iloc[i] = min(basic_lower_band.iloc[i], super_trend.iloc[i - 1])
    return super_trend

In [10]:
def calculate_monthly_indicators(df):
    df['SMA_20'] = talib.SMA(df['Close'], timeperiod=20)
    df['Choppiness_Index'] = calculate_choppiness_index(df)
    df['SuperTrend'] = calculate_super_trend(df)
    return df[['SMA_20', 'Choppiness_Index', 'SuperTrend']]

def calculate_weekly_indicators(df):
    df['MACD'], df['MACD_Signal'], _ = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    df['RSI'] = talib.RSI(df['Close'], timeperiod=14)
    df['SMA_50'] = talib.SMA(df['Close'], timeperiod=50)
    df['EMA_20'] = talib.EMA(df['Close'], timeperiod=20)
    df['EMA_10'] = talib.EMA(df['Close'], timeperiod=10)
    return df[['MACD', 'MACD_Signal', 'RSI', 'SMA_50', 'EMA_20', 'EMA_10']]

def calculate_daily_indicators(df):
    df['VWSMA_200'] = df['Close'].rolling(window=200).mean()
    df['VWEMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
    df['VWEMA_20'] = df['Close'].ewm(span=20, adjust=False).mean()
    df['RSI_Daily'] = talib.RSI(df['Close'], timeperiod=14)
    df['ADX'] = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod=14)
    return df[['VWSMA_200', 'VWEMA_50', 'VWEMA_20', 'RSI_Daily', 'ADX']]

In [11]:
process_stock('/home/tanishpatel01/Desktop/Stock_Market_Data/ADANIENT_Monthly.csv', '/home/tanishpatel01/Desktop/Stock_Market_Data/ADANIENT_Weekly.csv', '/home/tanishpatel01/Desktop/Stock_Market_Data/ADANIENT_Daily.csv', 'ADANIENT')

In [12]:
import pandas as pd
import talib
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def read_csv(file_path):
    df = pd.read_csv(file_path)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    return df

def calculate_monthly_indicators(df):
    df['SMA_20'] = talib.SMA(df['Close'], timeperiod=20)
    df['Choppiness_Index'] = calculate_choppiness_index(df)
    df['SuperTrend'] = calculate_super_trend(df)
    return df[['SMA_20', 'Choppiness_Index', 'SuperTrend']]

def calculate_weekly_indicators(df):
    df['MACD'], df['MACD_Signal'], _ = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    df['RSI'] = talib.RSI(df['Close'], timeperiod=14)
    df['SMA_50'] = talib.SMA(df['Close'], timeperiod=50)
    df['EMA_20'] = talib.EMA(df['Close'], timeperiod=20)
    df['EMA_10'] = talib.EMA(df['Close'], timeperiod=10)
    return df[['MACD', 'MACD_Signal', 'RSI', 'SMA_50', 'EMA_20', 'EMA_10']]

def calculate_daily_indicators(df):
    df['VWSMA_200'] = df['Close'].rolling(window=200).mean()
    df['VWEMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
    df['VWEMA_20'] = df['Close'].ewm(span=20, adjust=False).mean()
    df['RSI_Daily'] = talib.RSI(df['Close'], timeperiod=14)
    df['ADX'] = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod=14)
    return df[['VWSMA_200', 'VWEMA_50', 'VWEMA_20', 'RSI_Daily', 'ADX']]

def calculate_choppiness_index(df, period=14):
    hl_range = df['High'] - df['Low']
    atr = talib.SMA(talib.TRANGE(df['High'], df['Low'], df['Close']), period)
    normalized_range = hl_range / atr
    choppiness_index = talib.SMA(normalized_range, period)
    return choppiness_index

def calculate_super_trend(df, atr_period=14, multiplier=2):
    high = df['High']
    low = df['Low']
    close = df['Close']
    atr = talib.ATR(high, low, close, timeperiod=atr_period)
    basic_upper_band = (high + low) / 2 + multiplier * atr
    basic_lower_band = (high + low) / 2 - multiplier * atr
    super_trend = pd.Series(index=df.index)
    super_trend.iloc[0] = close.iloc[0]
    for i in range(1, len(super_trend)):
        if close.iloc[i - 1] <= basic_upper_band.iloc[i - 1]:
            super_trend.iloc[i] = max(basic_upper_band.iloc[i], super_trend.iloc[i - 1])
        else:
            super_trend.iloc[i] = min(basic_lower_band.iloc[i], super_trend.iloc[i - 1])
    return super_trend

def plot_trend(df_list, indicators, stock_name, timeframe):
    fig = make_subplots(rows=len(indicators), cols=1, shared_xaxes=True, 
                        subplot_titles=indicators)

    for i, indicator in enumerate(indicators):
        for df in df_list:
            fig.add_trace(go.Scatter(x=df.index, y=df[indicator], mode='lines', name=indicator), row=i+1, col=1)

    fig.update_layout(title=f'Stock: {stock_name} - {timeframe} Trends', 
                      xaxis_title='Date', 
                      yaxis_title='Indicator Value',
                      height=800, 
                      width=1000)
    fig.show()

def process_stock(monthly_file, weekly_file, daily_file, stock_name):
    df_monthly = read_csv(monthly_file)
    df_weekly = read_csv(weekly_file)
    df_daily = read_csv(daily_file)

    monthly_indicators = calculate_monthly_indicators(df_monthly)
    weekly_indicators = calculate_weekly_indicators(df_weekly)
    daily_indicators = calculate_daily_indicators(df_daily)

    plot_trend([monthly_indicators], ['SMA_20', 'Choppiness_Index', 'SuperTrend'], stock_name, 'Monthly')
    plot_trend([weekly_indicators], ['MACD', 'MACD_Signal', 'RSI', 'SMA_50', 'EMA_20', 'EMA_10'], stock_name, 'Weekly')
    plot_trend([daily_indicators], ['VWSMA_200', 'VWEMA_50', 'VWEMA_20', 'RSI_Daily', 'ADX'], stock_name, 'Daily')

# Example usage
process_stock('/home/tanishpatel01/Desktop/Stock_Market_Data/ADANIENT_Monthly.csv', '/home/tanishpatel01/Desktop/Stock_Market_Data/ADANIENT_Weekly.csv', '/home/tanishpatel01/Desktop/Stock_Market_Data/ADANIENT_Daily.csv', 'ADANIENT')


In [13]:
import pandas as pd
import talib
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def read_csv(file_path):
    df = pd.read_csv(file_path)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    return df

def calculate_monthly_indicators(df):
    df['SMA_20'] = talib.SMA(df['Close'], timeperiod=20)
    df['Choppiness_Index'] = calculate_choppiness_index(df)
    df['SuperTrend'] = calculate_super_trend(df)
    return df[['SMA_20', 'Choppiness_Index', 'SuperTrend']]

def calculate_weekly_indicators(df):
    df['MACD'], df['MACD_Signal'], _ = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    df['RSI'] = talib.RSI(df['Close'], timeperiod=14)
    df['SMA_50'] = talib.SMA(df['Close'], timeperiod=50)
    df['EMA_20'] = talib.EMA(df['Close'], timeperiod=20)
    df['EMA_10'] = talib.EMA(df['Close'], timeperiod=10)
    return df[['MACD', 'MACD_Signal', 'RSI', 'SMA_50', 'EMA_20', 'EMA_10']]

def calculate_daily_indicators(df):
    df['VWSMA_200'] = df['Close'].rolling(window=200).mean()
    df['VWEMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
    df['VWEMA_20'] = df['Close'].ewm(span=20, adjust=False).mean()
    df['RSI_Daily'] = talib.RSI(df['Close'], timeperiod=14)
    df['ADX'] = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod=14)
    df['MACD'], df['MACD_Signal'], _ = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    return df[['VWSMA_200', 'VWEMA_50', 'VWEMA_20', 'RSI_Daily', 'ADX', 'MACD', 'MACD_Signal']]

def calculate_choppiness_index(df, period=14):
    hl_range = df['High'] - df['Low']
    atr = talib.SMA(talib.TRANGE(df['High'], df['Low'], df['Close']), period)
    normalized_range = hl_range / atr
    choppiness_index = talib.SMA(normalized_range, period)
    return choppiness_index

def calculate_super_trend(df, atr_period=14, multiplier=2):
    high = df['High']
    low = df['Low']
    close = df['Close']
    atr = talib.ATR(high, low, close, timeperiod=atr_period)
    basic_upper_band = (high + low) / 2 + multiplier * atr
    basic_lower_band = (high + low) / 2 - multiplier * atr
    super_trend = pd.Series(index=df.index)
    super_trend.iloc[0] = close.iloc[0]
    for i in range(1, len(super_trend)):
        if close.iloc[i - 1] <= basic_upper_band.iloc[i - 1]:
            super_trend.iloc[i] = max(basic_upper_band.iloc[i], super_trend.iloc[i - 1])
        else:
            super_trend.iloc[i] = min(basic_lower_band.iloc[i], super_trend.iloc[i - 1])
    return super_trend

def process_stock(monthly_file, weekly_file, daily_file, stock_name):
    df_monthly = read_csv(monthly_file)
    df_weekly = read_csv(weekly_file)
    df_daily = read_csv(daily_file)

    monthly_indicators = calculate_monthly_indicators(df_monthly)
    weekly_indicators = calculate_weekly_indicators(df_weekly)
    daily_indicators = calculate_daily_indicators(df_daily)

    plot_trend(monthly_indicators, ['SMA_20', 'Choppiness_Index', 'SuperTrend'], stock_name, 'Monthly')
    plot_trend(weekly_indicators, ['MACD', 'RSI', 'SMA_50', 'EMA_20', 'EMA_10'], stock_name, 'Weekly')
    plot_trend(daily_indicators, ['VWSMA_200', 'VWEMA_50', 'VWEMA_20', 'RSI_Daily', 'ADX'], stock_name, 'Daily', ['MACD', 'MACD_Signal'])

def plot_trend(df, indicators, stock_name, timeframe, combined_indicators=None):
    fig = make_subplots(rows=len(indicators), cols=1, shared_xaxes=True, 
                        subplot_titles=indicators)

    for i, indicator in enumerate(indicators):
        if combined_indicators and indicator in combined_indicators:
            continue
        fig.add_trace(go.Scatter(x=df.index, y=df[indicator], mode='lines', name=indicator), row=i+1, col=1)

    if combined_indicators:
        combined_df = df[combined_indicators]
        fig.add_trace(go.Scatter(x=combined_df.index, y=combined_df['MACD'], mode='lines', name='MACD'), row=len(indicators)//2+1, col=1)
        fig.add_trace(go.Scatter(x=combined_df.index, y=combined_df['MACD_Signal'], mode='lines', name='MACD_Signal'), row=len(indicators)//2+1, col=1)

    fig.update_layout(title=f'Stock: {stock_name} - {timeframe} Trends', 
                      xaxis_title='Date', 
                      yaxis_title='Indicator Value',
                      height=800, 
                      width=1000)
    fig.show()


In [14]:
process_stock('/home/tanishpatel01/Desktop/Stock_Market_Data/ADANIENT_Monthly.csv', '/home/tanishpatel01/Desktop/Stock_Market_Data/ADANIENT_Weekly.csv', '/home/tanishpatel01/Desktop/Stock_Market_Data/ADANIENT_Daily.csv', 'ADANIENT')

In [12]:
import pandas as pd
from datetime import timedelta

script = 'ADANIENT'

# Load your datasets here
daily_data = pd.read_csv(f'/home/tanishpatel01/Documents/Backup_Tanish/Stock_Market_Data/Data_Visualisations/indicators_processed/{script}_daily_indicators_processed.csv')
weekly_data = pd.read_csv(f'/home/tanishpatel01/Documents/Backup_Tanish/Stock_Market_Data/Data_Visualisations/indicators_processed/{script}_weekly_indicators_processed.csv')
monthly_data = pd.read_csv(f'/home/tanishpatel01/Documents/Backup_Tanish/Stock_Market_Data/Data_Visualisations/indicators_processed/{script}_monthly_indicators_processed.csv')

In [14]:
# Redefine the helper functions first

def get_previous_week_data(date, weekly_data):
    start_of_week = date - timedelta(days=date.weekday())
    previous_week_start = start_of_week - timedelta(weeks=1)
    return weekly_data[weekly_data['Date'] == previous_week_start.strftime('%Y-%m-%d')]

def get_corresponding_month_data(date, monthly_data):
    start_of_month = date.replace(day=1)
    return monthly_data[monthly_data['Date'] == start_of_month.strftime('%Y-%m-%d')]

def check_rsi_trend(weekly_data, start_date, end_date, condition):
    weekly_data_filtered = weekly_data[(weekly_data['Date'] >= start_date.strftime('%Y-%m-%d')) & 
                                       (weekly_data['Date'] <= end_date.strftime('%Y-%m-%d'))]
    if weekly_data_filtered.empty or len(weekly_data_filtered) < 6:
        return False
    rsi_values = weekly_data_filtered['RSI']
    
    if condition == 'increasing_under_40':
        return all(rsi < 40 for rsi in rsi_values[:-1]) and rsi_values.iloc[-1] > 30 and all(x < y for x, y in zip(rsi_values, rsi_values[1:]))
    elif condition == 'increasing_50_to_80':
        return all(50 <= rsi <= 80 for rsi in rsi_values) and all(x < y for x, y in zip(rsi_values, rsi_values[1:]))
    elif condition == 'decreasing_above_70':
        return all(rsi > 70 for rsi in rsi_values) and all(x > y for x, y in zip(rsi_values, rsi_values[1:]))
    else:
        return False

# Redefine the main function with the helper functions

def calculate_buy_sell_tags(daily_data, weekly_data, monthly_data):
    buy_results = []
    sell_results = []

    for index, row in daily_data.iterrows():
        date = pd.to_datetime(row['Date'])
        weekly_row = get_previous_week_data(date, weekly_data)
        monthly_row = get_corresponding_month_data(date, monthly_data)

        if weekly_row.empty or monthly_row.empty:
            buy_results.append((row['Date'], 0))
            sell_results.append((row['Date'], 0))
            continue

        choppiness_index = monthly_row['Choppiness_Index'].iloc[0]
        supertrend = monthly_row['SuperTrend'].iloc[0]
        current_price = (row['Open'] + row['Close']) / 2
        sma_20 = monthly_row['SMA_20'].iloc[0]
        vwsma_200 = row['VWSMA_200']
        vwema_20 = row['VWEMA_20']
        vwema_50 = row['VWEMA_50']
        macd = weekly_row['MACD'].iloc[0]
        macd_signal = weekly_row['MACD_Signal'].iloc[0]
        rsi_weekly = weekly_row['RSI'].iloc[0]

        # Buy Logic
        if choppiness_index < 38.2:
            if supertrend > current_price:
                buy_results.append((row['Date'], 0))
            elif current_price > sma_20 and current_price > vwsma_200 and vwema_20 > vwema_50 and vwema_50 > vwsma_200:
                if macd > 0 and macd_signal > 0 and macd >= macd_signal:
                    if check_rsi_trend(weekly_data, date - timedelta(weeks=6), date, 'increasing_50_to_80'):
                        buy_results.append((row['Date'], 1))
                    else:
                        buy_results.append((row['Date'], 0))
                else:
                    buy_results.append((row['Date'], 0))
            else:
                buy_results.append((row['Date'], 0))
        elif choppiness_index > 61.8:
            if supertrend > current_price:
                buy_results.append((row['Date'], 0))
            elif current_price > sma_20 and current_price > vwsma_200 and vwema_20 > vwema_50 and vwema_50 > vwsma_200:
                if check_rsi_trend(weekly_data, date - timedelta(weeks=6), date, 'increasing_under_40'):
                    buy_results.append((row['Date'], 1))
                else:
                    buy_results.append((row['Date'], 0))
            else:
                buy_results.append((row['Date'], 0))
        else:
            buy_results.append((row['Date'], 0))

        # Sell Logic
        if check_rsi_trend(weekly_data, date - timedelta(weeks=6), date, 'decreasing_above_70'):
            if macd < macd_signal or current_price < sma_20 or current_price < vwsma_200 or vwema_20 < 0.95 * vwema_50:
                sell_results.append((row['Date'], 1))
            else:
                sell_results.append((row['Date'], 0))
        elif current_price < sma_20 or current_price < vwsma_200:
            sell_results.append((row['Date'], 1))
        else:
            sell_results.append((row['Date'], 0))

    return buy_results, sell_results

# Calculate buy and sell tags
buy_results, sell_results = calculate_buy_sell_tags(daily_data, weekly_data, monthly_data)

# Convert results to DataFrame and save
buy_df = pd.DataFrame(buy_results, columns=['Date', 'Buy_Tag'])
sell_df = pd.DataFrame(sell_results, columns=['Date', 'Sell_Tag'])
result_df = pd.merge(buy_df, sell_df, on='Date')
result_df.to_csv(f'{script}_updated_daily_data.csv', index=False)
